In [1]:
import pyfdb

In [29]:
import sys
import os
sys.path.append("../gsv_interface")

from gsv.retriever import GSVRetriever

gsv = GSVRetriever()  # Uncomment to request from YAML

In [3]:
request = { 'class': 'ea',
    'domain': 'g',
    'type': 'an',
    'stream': 'moda',
    'date': '19910401',
    'levelist': '500',
    'levtype': 'pl',
    'param': '129',
    'expver': '0001',
}
ds = gsv.request_data(request)

Correctly decoded z      on level 500    at 1991-04-01T00:00:00


In [35]:
request = { 
    'class': 'ea',
    'domain': 'g',
    'type': 'an',
    'stream': 'oper',
    'date': '20000401',
    'time': [ '0000','0100','0200','0300','0400','0500','0600','0700','0800','0900','1000','1100','1200','1300','1400','1500','1600','1700','1800','1900','2000','2100','2200','2300'],
    'levtype': 'sfc',
    'step': 0,
    'param': '167',
    'expver': '0001',
}
ds = gsv.request_data(request)
ds

Correctly decoded 2t     on level 0      at 2000-04-01T00:00:00
Correctly decoded 2t     on level 0      at 2000-04-01T01:00:00
Correctly decoded 2t     on level 0      at 2000-04-01T02:00:00
Correctly decoded 2t     on level 0      at 2000-04-01T03:00:00
Correctly decoded 2t     on level 0      at 2000-04-01T04:00:00
Correctly decoded 2t     on level 0      at 2000-04-01T05:00:00
Correctly decoded 2t     on level 0      at 2000-04-01T06:00:00
Correctly decoded 2t     on level 0      at 2000-04-01T07:00:00
Correctly decoded 2t     on level 0      at 2000-04-01T08:00:00
Correctly decoded 2t     on level 0      at 2000-04-01T09:00:00
Correctly decoded 2t     on level 0      at 2000-04-01T10:00:00
Correctly decoded 2t     on level 0      at 2000-04-01T11:00:00
Correctly decoded 2t     on level 0      at 2000-04-01T12:00:00
Correctly decoded 2t     on level 0      at 2000-04-01T13:00:00
Correctly decoded 2t     on level 0      at 2000-04-01T14:00:00
Correctly decoded 2t     on level 0     

<xarray.Dataset>
Dimensions:  (lon: 1440, lat: 721, height: 1, time: 24)
Coordinates:
  * lon      (lon) float64 0.0 0.25 0.5 0.75 1.0 ... 359.0 359.2 359.5 359.8
  * lat      (lat) float64 90.0 89.75 89.5 89.25 ... -89.25 -89.5 -89.75 -90.0
  * height   (height) int64 0
  * time     (time) datetime64[ns] 2000-04-01 ... 2000-04-01T23:00:00
Data variables:
    2t       (height, time, lat, lon) float64 246.2 246.2 246.2 ... 221.6 221.6

In [30]:
from datetime import datetime
from dateutil.relativedelta import relativedelta
import sys

def make_date_list(start, end, step="M", numsteps=1):

    if step == "M":
        delta = relativedelta(months=numsteps)
    elif step == "D":
        delta = relativedelta(days=numsteps)
    elif step == "H":
        delta = relativedelta(hours=numsteps)
    else:
        sys.exit("step not recognized")        

    start_date = datetime.strptime(start, '%Y%m%d')
    end_date = datetime.strptime(end, '%Y%m%d')

    date_list = []
    current_date = start_date

    while current_date <= end_date:
        date_string = current_date.strftime('%Y%m%d')
        date_list.append(date_string)
        current_date += delta

    return date_list

In [141]:
def gsv_generator(gsv, request, start, end):
    """Return GSV request as a generator from a date list"""
    date_list= make_date_list(start, end)
    for date in date_list:
        request["date"] = date
        yield gsv.request_data(request)
        



In [139]:
request = { 
    'class': 'ea',
    'domain': 'g',
    'type': 'an',
    'stream': 'oper',
    'date': '20110301',
    'time': [ '0000','0100','0200','0300','0400','0500','0600','0700','0800','0900','1000','1100','1200','1300','1400','1500','1600','1700','1800','1900','2000','2100','2200','2300'],
    'levtype': 'sfc',
    'step': 0,
    'param': '167',
    'expver': '0001',
}
gen = gsv_generator(gsv, request, '20101231', '20110104')

In [140]:
for ds in gen:
    print(ds)

Correctly decoded 2t     on level 0      at 2010-12-31T00:00:00
Correctly decoded 2t     on level 0      at 2010-12-31T01:00:00
Correctly decoded 2t     on level 0      at 2010-12-31T02:00:00
Correctly decoded 2t     on level 0      at 2010-12-31T03:00:00
Correctly decoded 2t     on level 0      at 2010-12-31T04:00:00
Correctly decoded 2t     on level 0      at 2010-12-31T05:00:00
Correctly decoded 2t     on level 0      at 2010-12-31T06:00:00
Correctly decoded 2t     on level 0      at 2010-12-31T07:00:00
Correctly decoded 2t     on level 0      at 2010-12-31T08:00:00
Correctly decoded 2t     on level 0      at 2010-12-31T09:00:00
Correctly decoded 2t     on level 0      at 2010-12-31T10:00:00
Correctly decoded 2t     on level 0      at 2010-12-31T11:00:00
Correctly decoded 2t     on level 0      at 2010-12-31T12:00:00
Correctly decoded 2t     on level 0      at 2010-12-31T13:00:00
Correctly decoded 2t     on level 0      at 2010-12-31T14:00:00
Correctly decoded 2t     on level 0     

In [125]:
from gsv import requests
req = requests.parse_request(request)
it=gsv._list(req)
print(len(list(it)))

In [135]:
len(list(it))

24

In [127]:

for dd in it:
    print(dd)


{class=ea,expver=0001,stream=oper,date=20000301,time=0000,domain=g}{type=an,levtype=sfc}{step=0,param=167}
{class=ea,expver=0001,stream=oper,date=20000301,time=0100,domain=g}{type=an,levtype=sfc}{step=0,param=167}
{class=ea,expver=0001,stream=oper,date=20000301,time=0200,domain=g}{type=an,levtype=sfc}{step=0,param=167}
{class=ea,expver=0001,stream=oper,date=20000301,time=0300,domain=g}{type=an,levtype=sfc}{step=0,param=167}
{class=ea,expver=0001,stream=oper,date=20000301,time=0400,domain=g}{type=an,levtype=sfc}{step=0,param=167}
{class=ea,expver=0001,stream=oper,date=20000301,time=0500,domain=g}{type=an,levtype=sfc}{step=0,param=167}
{class=ea,expver=0001,stream=oper,date=20000301,time=0600,domain=g}{type=an,levtype=sfc}{step=0,param=167}
{class=ea,expver=0001,stream=oper,date=20000301,time=0700,domain=g}{type=an,levtype=sfc}{step=0,param=167}
{class=ea,expver=0001,stream=oper,date=20000301,time=0800,domain=g}{type=an,levtype=sfc}{step=0,param=167}
{class=ea,expver=0001,stream=oper,dat

In [123]:
dd

<generator object ListIterator.__iter__ at 0x7f3403eb3bc0>

In [118]:
next(dd)

StopIteration: 

In [110]:
dd

'{class=ea,expver=0001,stream=oper,date=20000301,time=2300,domain=g}{type=an,levtype=sfc}{step=0,param=167}'

In [91]:
for ds in gsv._list(req):
    print(ds)

In [79]:
make_date_list('20101231', '20110103', step="D", numsteps=1)

['20101231', '20110101', '20110102', '20110103']

In [84]:
dd=next(gen)

Date 20110102 not found, waiting ...
Exception: Cannot open /work/users/jost/destine/fdb-root/ea:0001/20110102:0300:g/oper/schema  (No such file or directory)
Caught exception on C-C++ API boundary: Cannot open /work/users/jost/destine/fdb-root/ea:0001/20110102:0300:g/oper/schema  (No such file or directory)


FDBException: Error in function fdb_retrieve: Cannot open /work/users/jost/destine/fdb-root/ea:0001/20110102:0300:g/oper/schema  (No such file or directory)

In [72]:
for dd in gen:
    print(dd.time)

Correctly decoded 2t     on level 0      at 2010-12-31T00:00:00
Correctly decoded 2t     on level 0      at 2010-12-31T01:00:00
Correctly decoded 2t     on level 0      at 2010-12-31T02:00:00
Correctly decoded 2t     on level 0      at 2010-12-31T03:00:00
Correctly decoded 2t     on level 0      at 2010-12-31T04:00:00
Correctly decoded 2t     on level 0      at 2010-12-31T05:00:00
Correctly decoded 2t     on level 0      at 2010-12-31T06:00:00
Correctly decoded 2t     on level 0      at 2010-12-31T07:00:00
Correctly decoded 2t     on level 0      at 2010-12-31T08:00:00
Correctly decoded 2t     on level 0      at 2010-12-31T09:00:00
Correctly decoded 2t     on level 0      at 2010-12-31T10:00:00
Correctly decoded 2t     on level 0      at 2010-12-31T11:00:00
Correctly decoded 2t     on level 0      at 2010-12-31T12:00:00
Correctly decoded 2t     on level 0      at 2010-12-31T13:00:00
Correctly decoded 2t     on level 0      at 2010-12-31T14:00:00
Correctly decoded 2t     on level 0     

In [142]:
import time
from gsv import requests

def gsv_generator(gsv, request, start, end):
    """Return GSV request as a generator from a date list"""
    date_list= make_date_list(start, end, step='D')
    for date in date_list:
        request["date"] = date

        req = requests.parse_request(request)

        ds = gsv.request_data(request)
        if not ds:
            print(f"Date {date} not found, waiting ...")
        while not ds:
            time.sleep(1)
            it=list(gsv._list(req))
            if len(it):
                print(it)
                ds = gsv.request_data(request)
        yield ds
        
        


In [143]:
request = { 
    'class': 'ea',
    'domain': 'g',
    'type': 'an',
    'stream': 'oper',
    'date': '20110301',
    'time': [ '0000','0100','0200','0300','0400','0500','0600','0700','0800','0900','1000','1100','1200','1300','1400','1500','1600','1700','1800','1900','2000','2100','2200','2300'],
    'levtype': 'sfc',
    'step': 0,
    'param': '167',
    'expver': '0001',
}
gen = gsv_generator(gsv, request, '20101231', '20110105')

In [149]:
ds = next(gen)

Date 20110105 not found, waiting ...
['{class=ea,expver=0001,stream=oper,date=20110105,time=0000,domain=g}{type=an,levtype=sfc}{step=0,param=167}', '{class=ea,expver=0001,stream=oper,date=20110105,time=0100,domain=g}{type=an,levtype=sfc}{step=0,param=167}', '{class=ea,expver=0001,stream=oper,date=20110105,time=0200,domain=g}{type=an,levtype=sfc}{step=0,param=167}', '{class=ea,expver=0001,stream=oper,date=20110105,time=0300,domain=g}{type=an,levtype=sfc}{step=0,param=167}', '{class=ea,expver=0001,stream=oper,date=20110105,time=0400,domain=g}{type=an,levtype=sfc}{step=0,param=167}', '{class=ea,expver=0001,stream=oper,date=20110105,time=0500,domain=g}{type=an,levtype=sfc}{step=0,param=167}', '{class=ea,expver=0001,stream=oper,date=20110105,time=0600,domain=g}{type=an,levtype=sfc}{step=0,param=167}', '{class=ea,expver=0001,stream=oper,date=20110105,time=0700,domain=g}{type=an,levtype=sfc}{step=0,param=167}', '{class=ea,expver=0001,stream=oper,date=20110105,time=0800,domain=g}{type=an,levty

In [52]:
request["date"] = "20000101"
ds3 = gsv.request_data(request)

Correctly decoded 2t     on level 0      at 2000-01-01T00:00:00
Correctly decoded 2t     on level 0      at 2000-01-01T01:00:00
Correctly decoded 2t     on level 0      at 2000-01-01T02:00:00
Correctly decoded 2t     on level 0      at 2000-01-01T03:00:00
Correctly decoded 2t     on level 0      at 2000-01-01T04:00:00
Correctly decoded 2t     on level 0      at 2000-01-01T05:00:00
Correctly decoded 2t     on level 0      at 2000-01-01T06:00:00
Correctly decoded 2t     on level 0      at 2000-01-01T07:00:00
Correctly decoded 2t     on level 0      at 2000-01-01T08:00:00
Correctly decoded 2t     on level 0      at 2000-01-01T09:00:00
Correctly decoded 2t     on level 0      at 2000-01-01T10:00:00
Correctly decoded 2t     on level 0      at 2000-01-01T11:00:00
Correctly decoded 2t     on level 0      at 2000-01-01T12:00:00
Correctly decoded 2t     on level 0      at 2000-01-01T13:00:00
Correctly decoded 2t     on level 0      at 2000-01-01T14:00:00
Correctly decoded 2t     on level 0     

In [53]:
if ds3:
    print("yes")

yes


In [64]:
for ds in gen:
    print(ds)
    

In [ ]:
# https://github.com/intake/intake-examples/blob/master/tutorial/data_engineer.ipynb

In [28]:
import intake

In [29]:
list(intake.registry)

['netcdf',
 'opendap',
 'rasterio',
 'remote-xarray',
 'xarray_image',
 'zarr',
 'alias',
 'catalog',
 'csv',
 'intake_remote',
 'json',
 'jsonl',
 'ndzarr',
 'numpy',
 'textfiles',
 'tiled',
 'tiled_cat',
 'yaml_file_cat',
 'yaml_files_cat',
 'zarr_cat',
 'esm_datastore',
 'esm_group',
 'esm_single_source']

In [30]:
source = intake.open_csv('https://timeseries.weebly.com/uploads/'
                         '2/1/0/8/21086414/sea_ice.csv')

In [32]:
# access the data basic details
source.discover()

{'dtype': {'Time': 'object', 'Arctic': 'float64', 'Antarctica': 'float64'},
 'shape': (None, 3),
 'npartitions': 1,
 'metadata': {}}

In [33]:
# load and use data
df = source.read()
df.head()

,Time,Arctic,Antarctica
0,1990M01,12.72,3.27
1,1990M02,13.33,2.15
2,1990M03,13.44,2.71
3,1990M04,12.16,5.10
4,1990M05,10.84,7.37


In [34]:
# presciption for the source we made
print(source.yaml())

sources:
  csv:
    args:
      urlpath: https://timeseries.weebly.com/uploads/2/1/0/8/21086414/sea_ice.csv
    description: ''
    driver: intake.source.csv.CSVSource
    metadata: {}



In [7]:
from datetime import datetime
from dateutil.relativedelta import relativedelta
import sys

def make_date_list(start, end, step="M", numsteps=1):

    if step == "M":
        delta = relativedelta(months=numsteps)
    elif step == "D":
        delta = relativedelta(days=numsteps)
    elif step == "H":
        delta = relativedelta(hours=numsteps)
    else:
        sys.exit("step not recognized")        

    start_date = datetime.strptime(start, '%Y%m%d')
    end_date = datetime.strptime(end, '%Y%m%d')

    date_list = []
    current_date = start_date

    while current_date <= end_date:
        date_string = current_date.strftime('%Y%m%d')
        date_list.append(date_string)
        current_date += delta

    return date_list

In [4]:
import os
import sys
sys.path.append("../gsv_interface")

from gsv.retriever import GSVRetriever
import xarray as xr
from intake.source import base

class GSVSource(base.DataSource):
    container = xr.Dataset
    name = 'gsv'
    version = '0.0.1'
    partition_access = True

    def __init__(self, request, startdate="20200101", enddate="20201201", metadata=None, **kwargs):
        self._request = request
        self._kwargs = kwargs
        self._dates = make_date_list(startdate, enddate)
        self._npartitions = len(self._dates)
        self.gsv = GSVRetriever()
        super(GSVSource, self).__init__(metadata=metadata)

    def _get_schema(self):
        # if self._dataset is None:
        #     self._load()
        return base.Schema(
            datashape=None,
            dtype=xr.Dataset,
            #shape=self._dataset.shape,
            shape=None,
            npartitions=self._npartitions,
            extra_metadata={},
        )

    def _get_partition(self, i):
        self._request["date"] = self._dates[i]
        dataset = self.gsv.request_data(self._request)
        return dataset

    def read(self):
        ds = [self._get_partition(i) for i in range(self._npartitions)]
        ds = xr.concat(ds, dim='time')
        return ds
    
    # def _load(self):
    #     self._dataset = self._get_partition(0)

In [2]:
import os
import sys
import xarray as xr
from intake.source import base

sys.path.append("../gsv_interface")
from gsv.retriever import GSVRetriever

class GSVSource(base.DataSource):
    container = 'xarray'
    name = 'gsv'
    version = '0.0.1'
    partition_access = True

    def __init__(self, request, startdate="20200101", enddate="20201201", var='167', metadata=None, **kwargs):
        self._request = request
        self._kwargs = kwargs
        self._dates = make_date_list(startdate, enddate)
        self._var = var
        self._npartitions = len(self._dates)
        self.gsv = GSVRetriever()
        self._dataset = None
        super(GSVSource, self).__init__(metadata=metadata)

    def _get_schema(self):
        # if self._dataset is None:
        #     self._get_partition(0)
        return base.Schema(
            datashape=None,
            dtype=xr.Dataset,
            #shape=self._dataset.shape,
            shape=None,
            npartitions=self._npartitions,
            extra_metadata={},
        )

    def _get_partition(self, i):
        self._request["date"] = self._dates[i]
        self._request["param"] = self._var
        dataset = self.gsv.request_data(self._request)
        return dataset

    def read(self):
        ds = [self._get_partition(i) for i in range(self._npartitions)]
        ds = xr.concat(ds, dim='time')
        return ds
    
    def to_dask(self):
        return self.read()
    
    # def _load(self):
    #     self._dataset = self._get_partition(0)

In [1]:
import intake
import aqua.gsv

In [2]:
# import imp
# imp.reload(intake)

In [3]:
import intake
intake.registry.drivers.register_driver('gsv', GSVSource)

In [36]:
list(intake.registry)

['netcdf',
 'opendap',
 'rasterio',
 'remote-xarray',
 'xarray_image',
 'zarr',
 'alias',
 'catalog',
 'csv',
 'intake_remote',
 'json',
 'jsonl',
 'ndzarr',
 'numpy',
 'textfiles',
 'tiled',
 'tiled_cat',
 'yaml_file_cat',
 'yaml_files_cat',
 'zarr_cat',
 'esm_datastore',
 'esm_group',
 'esm_single_source',
 'gsv',
 'daily']

In [8]:
import intake
import aqua.gsv
cat = intake.open_catalog("config/mafalda/catalog.yaml")
ds = cat["ERA5"]["fdb"]["daily"](startdate="20000101", enddate="20000102", var='167')
dd = ds.read()


Correctly decoded 2t     on level 0      at 2000-01-01T00:00:00
Correctly decoded 2t     on level 0      at 2000-01-01T01:00:00
Correctly decoded 2t     on level 0      at 2000-01-01T02:00:00
Correctly decoded 2t     on level 0      at 2000-01-01T03:00:00
Correctly decoded 2t     on level 0      at 2000-01-01T04:00:00
Correctly decoded 2t     on level 0      at 2000-01-01T05:00:00
Correctly decoded 2t     on level 0      at 2000-01-01T06:00:00
Correctly decoded 2t     on level 0      at 2000-01-01T07:00:00
Correctly decoded 2t     on level 0      at 2000-01-01T08:00:00
Correctly decoded 2t     on level 0      at 2000-01-01T09:00:00
Correctly decoded 2t     on level 0      at 2000-01-01T10:00:00
Correctly decoded 2t     on level 0      at 2000-01-01T11:00:00
Correctly decoded 2t     on level 0      at 2000-01-01T12:00:00
Correctly decoded 2t     on level 0      at 2000-01-01T13:00:00
Correctly decoded 2t     on level 0      at 2000-01-01T14:00:00
Correctly decoded 2t     on level 0     

In [17]:
isinstance(cat["ERA5"]["fdb"]["daily"] , aqua.gsv.intake_gsv.GSVSource)

True

In [7]:
dd = ds.read()

Correctly decoded 2t     on level 0      at 2000-01-01T00:00:00
Correctly decoded 2t     on level 0      at 2000-01-01T01:00:00
Correctly decoded 2t     on level 0      at 2000-01-01T02:00:00
Correctly decoded 2t     on level 0      at 2000-01-01T03:00:00
Correctly decoded 2t     on level 0      at 2000-01-01T04:00:00
Correctly decoded 2t     on level 0      at 2000-01-01T05:00:00
Correctly decoded 2t     on level 0      at 2000-01-01T06:00:00
Correctly decoded 2t     on level 0      at 2000-01-01T07:00:00
Correctly decoded 2t     on level 0      at 2000-01-01T08:00:00
Correctly decoded 2t     on level 0      at 2000-01-01T09:00:00
Correctly decoded 2t     on level 0      at 2000-01-01T10:00:00
Correctly decoded 2t     on level 0      at 2000-01-01T11:00:00
Correctly decoded 2t     on level 0      at 2000-01-01T12:00:00
Correctly decoded 2t     on level 0      at 2000-01-01T13:00:00
Correctly decoded 2t     on level 0      at 2000-01-01T14:00:00
Correctly decoded 2t     on level 0     

In [6]:
dd

NameError: name 'dd' is not defined

In [7]:
d = ds.to_dask()

NotImplementedError: 

In [23]:
from aqua import Reader, catalogue
reader = Reader(model="ERA5", exp="fdb", source="monthly")
data = reader.retrieve_stream(fix=True, startdate='20010101', enddate='20011201', var='167')

In [28]:
dd=next(data)

Correctly decoded 2t     on level 0      at 2001-05-01T00:00:00


In [19]:
dd.time

<xarray.DataArray 'time' (time: 1)>
array(['2001-02-01T00:00:00.000000000'], dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 2001-02-01

In [25]:
ds = intake.open_gsv(request)

In [16]:
import xarray as xr
ds2 = xr.open_dataset('/work/users/jost/era5/day/era5.t2m.day.2000.grib', engine='cfgrib')

In [22]:
dd

<xarray.Dataset>
Dimensions:  (lon: 1440, lat: 721, height: 1, time: 1)
Coordinates:
  * lon      (lon) float64 0.0 0.25 0.5 0.75 1.0 ... 359.0 359.2 359.5 359.8
  * lat      (lat) float64 90.0 89.75 89.5 89.25 ... -89.25 -89.5 -89.75 -90.0
  * height   (height) int64 0
  * time     (time) datetime64[ns] 2001-01-01
Data variables:
    2t       (height, time, lat, lon) float64 248.5 248.5 248.5 ... 245.1 245.1

In [17]:
ds2

<xarray.Dataset>
Dimensions:     (time: 8784, latitude: 721, longitude: 1440)
Coordinates:
    number      int64 ...
  * time        (time) datetime64[ns] 2000-01-01 ... 2000-12-31T23:00:00
    step        timedelta64[ns] ...
    surface     float64 ...
  * latitude    (latitude) float64 90.0 89.75 89.5 89.25 ... -89.5 -89.75 -90.0
  * longitude   (longitude) float64 0.0 0.25 0.5 0.75 ... 359.2 359.5 359.8
    valid_time  (time) datetime64[ns] ...
Data variables:
    t2m         (time, latitude, longitude) float32 ...
Attributes:
    GRIB_edition:            1
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2023-03-29T18:15 GRIB to CDM+CF via cfgrib-0.9.1...

In [8]:
dd = ds.read_chunked()

In [10]:
for d in dd:
    print(d)

In [11]:
d

<xarray.Dataset>
Dimensions:  ()
Data variables:
    *empty*

In [4]:
from intake import open_catalog

cat = open_catalog("https://raw.githubusercontent.com/pangeo-data/pangeo-datastore/master/intake-catalogs/master.yaml")

In [10]:
cat['atmosphere']['era5_hourly_reanalysis_single_levels_sa']


era5_hourly_reanalysis_single_levels_sa:
  args:
    consolidated: true
    storage_options:
      requester_pays: true
    urlpath: gs://pangeo-era5/reanalysis/spatial-analysis
  description: ERA5 hourly estimates of variables on single levels
  driver: intake_xarray.xzarr.ZarrSource
  metadata:
    catalog_dir: https://raw.githubusercontent.com/pangeo-data/pangeo-datastore/master/intake-catalogs
    tags:
    - ocean
    - model
    - atmosphere
    url: https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-single-levels
